# OpenR1 Qwen2.5-0.5B-gsm8k-cppo
* num_generations: 8
* num_train_epochs: 1

https://github.com/lzhxmu/CPPO.git


- gpu: L4*1(Colab)
- model: Qwen/Qwen2.5-0.5B
- data: stpete2/openai-gsm8k-part-100
- method: cppo
- output: Qwen2.5-0.5B-gsm8k-cppo

###### unique setting for cppo in custom_config2.yaml
- metric: smallest
- pruning: 0.5
- allocation: true

## Open-R1
is an open initiative to replicate and extend the techniques behind DeepSeek-R1, a state-of-the-art reasoning model, in a fully transparent and collaborative way:

https://github.com/huggingface/open-r1



By selecting the model, dataset, and method, and running the training command from the command line, we were able to successfully perform training using the OpenR1 environment.

Cconsidering the limitations of the notebook environment, I limited the model and data to a minimum. And the following techniques are used.

* 1. Using LoRA (Low-Rank Adaptation)
* 2. Gradient checkpointing
* 3. Batching optimizations
* 4. BF16 mixed precision
* 5. Sequence length limit
* 6. Data packing

This setting is far from sufficient for effective training, but on the other hand, it allows us to check the operation of the method in a short time.

This minimal configuration allows for rapid validation of the training pipeline even with limited resources, and is a useful starting point before scaling up to larger experiments.

In [1]:
import wandb
secret_value = "xxxxx"
wandb.login(key=secret_value)

# save metrics into wandb folder
import os
os.environ["WANDB_DIR"] = "./wandb"
wandb.init(project="250424cppo", mode="online")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: stpeteishii to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [2]:
!git clone https://github.com/lzhxmu/CPPO.git
!pip install -e ./CPPO
!pip show CPPO

Cloning into 'CPPO'...
remote: Enumerating objects: 258, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 258 (delta 48), reused 58 (delta 39), pack-reused 184 (from 1)
Receiving objects: 100% (258/258), 4.15 MiB | 8.77 MiB/s, done.
Resolving deltas: 100% (113/113), done.
Obtaining file:///content/CPPO
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/huggingface/trl.git (to revision 69ad852e5654a77f1695eb4c608906fe0c7e8624) to /tmp/pip-install-igq2oila/trl_d202c3dc9f9348528d4698a20240a240
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl.git /tmp/pip-install-igq2oila/trl_d202c3dc9f9348528d4698a20240a240
  Running command git rev-parse -q --verify 'sha^69ad852e5654a77f1695eb4c608906fe0c7e8624'
  Running command git fetch -q https://github.com/huggingface/trl.git 69ad852e5654a77f1695eb4c608906fe0c7e8624
  Running command git checkout -q 69ad852e5654a77f1695eb4c60

In [3]:
import os
os.chdir('./CPPO')

In [4]:
!ls

LICENSE  Makefile  README.md  recipes  scripts	setup.cfg  setup.py  src


In [5]:
!pip install flash-attn --no-build-isolation
#!pip install vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 71.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.7.4.post1-cp311-cp311-linux_x86_64.whl size=187831595 sha256=58853b28a5a926cae14402bfd8d4d93a45ebf8f9e79533f37ab09d0d77a99c05
  Stored in directory: /root/.cache/pip/wheels/3d/88/d8/284b89f56af7d5bf366b10d6b8e251ac8a7c7bf3f04203fb4f
Successfully built flash-attn


In [6]:
from pathlib import Path


config_content = """
compute_environment: LOCAL_MACHINE
debug: false

deepspeed_config:
  gradient_clipping: 1.0
  zero3_init_flag: true
  zero_stage: 1
distributed_type: no

downcast_bf16: 'no'
machine_rank: 0
main_training_function: main
mixed_precision: bf16
num_machines: 1
num_processes: 1
rdzv_backend: static
same_network: true
tpu_env: []
tpu_use_cluster: false
tpu_use_sudo: false
use_cpu: false
"""

config_path = "custom_config.yaml"
Path(config_path).write_text(config_content)


#################################


config_content2 = """
# Model arguments
model_name_or_path: stpete2/Qwen2.5-0.5B-gsm8k-sft
model_revision: main
torch_dtype: bfloat16
attn_implementation: eager

# Data training arguments
dataset_name: stpete2/openai-gsm8k-part
system_prompt: |
  You are a helpful AI Assistant that provides well-reasoned and detailed responses. You first think about the reasoning process as an internal monologue and then provide the user with the answer. Respond in the following format: <think>
  ...
  </think>
  <answer>
  ...
  </answer>
# GRPO trainer config
bf16: true
use_vllm: false
do_eval: false
gradient_accumulation_steps: 2
gradient_checkpointing: true
gradient_checkpointing_kwargs:
  use_reentrant: false
hub_model_id: Qwen2.5-0.5B-gsm8k-cppo
hub_strategy: every_save
learning_rate: 2.0e-06
log_completions: true
log_level: info
logging_first_step: true
logging_steps: 1
logging_strategy: steps
lr_scheduler_type: cosine
max_prompt_length: 256
max_completion_length: 512
max_steps: -1

num_generations: 8
num_train_epochs: 1

metric: smallest
pruning: 0.75
allocation: true

output_dir: data/Qwen2.5-0.5B-gsm8k-cppo
overwrite_output_dir: true
per_device_eval_batch_size: 16
per_device_train_batch_size: 8
push_to_hub: false
report_to:
- wandb
reward_funcs:
- accuracy
- format
- tag_count
reward_weights:
- 1.0
- 1.0
- 1.0
save_strategy: "epoch"
save_total_limit: 1
seed: 42
warmup_ratio: 0.1
"""

config_path2 = "custom_config2.yaml"
Path(config_path2).write_text(config_content2)


##########################################################


!accelerate launch --config_file custom_config.yaml src/open_r1/grpo_gsm.py \
--config custom_config2.yaml \
--disable_tqdm=False



2025-05-07 03:07:46.776405: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-07 03:07:46.794249: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746587266.815918    1694 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746587266.822929    1694 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-07 03:07:46.844741: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [7]:
import shutil
from google.colab import files
folder_path = "data"
zip_filename = "data.zip"
shutil.make_archive(folder_path, 'zip', folder_path)
files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>